# 3.9 多层感知机的从零开始实现

In [ ]:
import torch
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
import d2lzh_pytorch as d2l

print(torch.__version__)

## 3.9.1 获取和读取数据

In [ ]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 3.9.2 定义模型参数

In [ ]:
def get_current_device():
    if torch.cuda.is_available() :
        device = torch.device('cuda')
        gpu_index = torch.cuda.current_device()
        current_device = str(device) + ':' + str(gpu_index)
    else :
        device = torch.device('cpu')
        current_device = str(device)
    return current_device

In [ ]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

W1 = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_hiddens)), dtype=torch.float)
b1 = torch.zeros(num_hiddens, dtype=torch.float)
W2 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens, num_outputs)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = get_current_device()
print(device)

print(W1.device, W2.device, b1.device, b2.device) #查看W1在cpu还是gpu
W1 = W1.to(device)
W2 = W2.to(device)
b1 = b1.to(device)
b2 = b2.to(device)

print(W1.device, W2.device, b1.device, b2.device)

params = [W1, b1, W2, b2]
for param in params:
    param.requires_grad_(requires_grad=True)

print(param.device)

## 3.9.3 定义激活函数

In [ ]:
def relu(X):
    X = X.to(device)
    # print(X.device)
    other = torch.tensor(0.0)
    other = other.to(device)
    return torch.max(input=X, other=other)

## 3.9.4 定义模型

In [ ]:
def net(X):
    X = X.to(device)
    X = X.view((-1, num_inputs))
    H = relu(torch.matmul(X, W1) + b1)
    return torch.matmul(H, W2) + b2

## 3.9.5 定义损失函数

In [ ]:
loss = torch.nn.CrossEntropyLoss()

## 3.9.6 训练模型

In [ ]:
def sgd(params, lr, batch_size):
    # 为了和原书保持一致，这里除以了batch_size，但是应该是不用除的，因为一般用PyTorch计算loss时就默认已经
    # 沿batch维求了平均了。
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

In [ ]:
def evaluate_accuracy(data_iter, net, 
        device = get_current_device()):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            # 这里加入2句,使得X,y自适应当前的device
            X = X.to(device)
            y = y.to(device)

            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

In [ ]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    device = get_current_device()
    print("training on ", device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            # 这里加入2句,使得X,y自适应当前的device
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到
            
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [ ]:
num_epochs, lr = 5, 100.0
from time import time
start = time()
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)
print(time() - start)
#train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)